# to_dict - Universal Dictionary Conversion

`to_dict` is a comprehensive utility for converting various Python objects to dictionaries. It provides:

**Core Features:**
- **Universal Conversion**: Handles Pydantic models, dataclasses, enums, sets, mappings, iterables, and custom objects
- **Recursive Processing**: Deep conversion of nested structures with depth control
- **JSON String Parsing**: Auto-parse JSON strings with fuzzy parsing support
- **Flexible Modes**: Prioritize Pydantic `.model_dump()`, control enum serialization, custom parsers
- **Error Handling**: Suppress exceptions and return empty dict on failure
- **Type Preservation**: Maintains container types during recursive processing

In [1]:
from dataclasses import dataclass
from enum import Enum

from lionherd_core.ln import to_dict

## 1. Basic Type Conversions

Convert various Python types to dictionaries.

In [2]:
# Dict → dict (copy)
source = {"name": "Alice", "age": 30}
result = to_dict(source)
print(f"Dict: {result}")
print(f"Is copy: {result is not source}")

Dict: {'name': 'Alice', 'age': 30}
Is copy: True


In [3]:
# List → enumerated dict {index: value}
items = ["apple", "banana", "cherry"]
result = to_dict(items)
print(f"List: {result}")

List: {0: 'apple', 1: 'banana', 2: 'cherry'}


In [4]:
# Tuple → enumerated dict
coords = (10, 20, 30)
result = to_dict(coords)
print(f"Tuple: {result}")

Tuple: {0: 10, 1: 20, 2: 30}


In [5]:
# Set → {value: value} mapping
tags = {"python", "tutorial", "demo"}
result = to_dict(tags)
print(f"Set: {result}")

Set: {'demo': 'demo', 'tutorial': 'tutorial', 'python': 'python'}


In [6]:
# None → empty dict
result = to_dict(None)
print(f"None: {result}")

None: {}


## 2. JSON String Parsing

Automatically parse JSON strings into Python dictionaries.

In [7]:
# Valid JSON string
json_str = '{"name": "Bob", "age": 25, "city": "NYC"}'
result = to_dict(json_str)
print(f"Parsed JSON: {result}")
print(f"Type: {type(result)}")

Parsed JSON: {'name': 'Bob', 'age': 25, 'city': 'NYC'}
Type: <class 'dict'>


In [8]:
# Fuzzy parsing for malformed JSON
malformed = '{name: "Bob", age: 25, city: "NYC"}'
result = to_dict(malformed, fuzzy_parse=True)
print(f"Fuzzy parsed: {result}")

Fuzzy parsed: {'name': 'Bob', 'age': 25, 'city': 'NYC'}


In [9]:
# Non-JSON string → kept as-is if parsing fails
non_json = "just a plain string"
result = to_dict(non_json, suppress=True)  # suppress errors
print(f"Non-JSON (suppressed): {result}")

Non-JSON (suppressed): {}


## 3. Dataclass Support

Convert dataclasses to dictionaries using `dataclasses.asdict()`.

In [10]:
@dataclass
class Person:
    name: str
    age: int
    city: str = "Unknown"


person = Person(name="Alice", age=30, city="SF")
result = to_dict(person)
print(f"Dataclass: {result}")

Dataclass: {'name': 'Alice', 'age': 30, 'city': 'SF'}


In [11]:
# Nested dataclass
@dataclass
class Address:
    street: str
    city: str


@dataclass
class Employee:
    name: str
    address: Address


emp = Employee(name="Bob", address=Address(street="123 Main St", city="NYC"))
result = to_dict(emp)
print(f"Nested dataclass: {result}")

Nested dataclass: {'name': 'Bob', 'address': {'street': '123 Main St', 'city': 'NYC'}}


## 4. Enum Handling

Convert enum classes and instances with control over value serialization.

In [12]:
class Status(Enum):
    PENDING = "pending"
    ACTIVE = "active"
    COMPLETED = "completed"


# Enum class → members dict (name → enum)
result_enum = to_dict(Status, use_enum_values=False)
print(f"Enum class (members): {result_enum}")
print(f"PENDING type: {type(result_enum['PENDING'])}")

Enum class (members): {'PENDING': <Status.PENDING: 'pending'>, 'ACTIVE': <Status.ACTIVE: 'active'>, 'COMPLETED': <Status.COMPLETED: 'completed'>}
PENDING type: <enum 'Status'>


In [13]:
# Enum class → members dict (name → value)
class Status(Enum):
    PENDING = "pending"
    ACTIVE = "active"
    COMPLETED = "completed"


result_values = to_dict(Status, use_enum_values=True)
print(f"Enum class (values): {result_values}")
print(f"PENDING type: {type(result_values['PENDING'])}")

Enum class (values): {'PENDING': 'pending', 'ACTIVE': 'active', 'COMPLETED': 'completed'}
PENDING type: <class 'str'>


## 5. Custom Objects with Methods

Handle objects with `.to_dict()`, `.dict()`, `.model_dump()`, `.to_json()`, or `__dict__` attributes.

In [14]:
# Object with .to_dict() method
class CustomObj:
    def __init__(self, name: str, value: int):
        self.name = name
        self.value = value

    def to_dict(self) -> dict:
        return {"name": self.name, "value": self.value}


obj = CustomObj("test", 42)
result = to_dict(obj)
print(f"Custom object with .to_dict(): {result}")

Custom object with .to_dict(): {'name': 'test', 'value': 42}


In [15]:
# Object with __dict__
class SimpleObj:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y


simple = SimpleObj(10, 20)
result = to_dict(simple)
print(f"Object with __dict__: {result}")

Object with __dict__: {'x': 10, 'y': 20}


## 6. Recursive Processing

Deep conversion of nested structures with depth control.

In [16]:
# Nested structure with JSON strings
nested = {
    "user": '{"name": "Alice", "age": 30}',
    "tags": '["python", "tutorial"]',
    "metadata": {"created": '{"timestamp": 1234567890}'},
}

# Without recursion - JSON strings remain strings
result_no_recurse = to_dict(nested, recursive=False)
print(f"No recursion: {result_no_recurse}")
print(f"user type: {type(result_no_recurse['user'])}")

No recursion: {'user': '{"name": "Alice", "age": 30}', 'tags': '["python", "tutorial"]', 'metadata': {'created': '{"timestamp": 1234567890}'}}
user type: <class 'str'>


In [17]:
# With recursion - JSON strings parsed
nested = {
    "user": '{"name": "Alice", "age": 30}',
    "tags": '["python", "tutorial"]',
    "metadata": {"created": '{"timestamp": 1234567890}'},
}

result_recurse = to_dict(nested, recursive=True)
print(f"With recursion: {result_recurse}")
print(f"user type: {type(result_recurse['user'])}")
print(f"user content: {result_recurse['user']}")

With recursion: {'user': {'name': 'Alice', 'age': 30}, 'tags': ['python', 'tutorial'], 'metadata': {'created': {'timestamp': 1234567890}}}
user type: <class 'dict'>
user content: {'name': 'Alice', 'age': 30}


In [18]:
# Depth control
deep_nested = {"level1": {"level2": {"level3": {"level4": {"level5": {"value": "deep"}}}}}}

result_depth_3 = to_dict(deep_nested, recursive=True, max_recursive_depth=3)
print(f"Max depth 3: {result_depth_3}")

Max depth 3: {'level1': {'level2': {'level3': {'level4': {'level5': {'value': 'deep'}}}}}}


## 7. Recursive Custom Types

Control whether custom objects are converted during recursive processing.

In [19]:
@dataclass
class Point:
    x: int
    y: int


@dataclass
class Shape:
    name: str
    points: list


shape = Shape(name="triangle", points=[Point(0, 0), Point(1, 0), Point(0, 1)])

# recursive_python_only=True (default) - custom types NOT converted during recursion
result_python_only = to_dict(shape, recursive=True, recursive_python_only=True)
print(f"Python only: {result_python_only}")
print(f"points[0] type: {type(result_python_only['points'][0])}")

Python only: {'name': 'triangle', 'points': [{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 0, 'y': 1}]}
points[0] type: <class 'dict'>


In [20]:
# recursive_python_only=False - custom types converted recursively
@dataclass
class Point:
    x: int
    y: int


@dataclass
class Shape:
    name: str
    points: list


shape = Shape(name="triangle", points=[Point(0, 0), Point(1, 0), Point(0, 1)])

result_all = to_dict(shape, recursive=True, recursive_python_only=False)
print(f"All types: {result_all}")
print(f"points[0] type: {type(result_all['points'][0])}")
print(f"points[0]: {result_all['points'][0]}")

All types: {'name': 'triangle', 'points': [{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 0, 'y': 1}]}
points[0] type: <class 'dict'>
points[0]: {'x': 0, 'y': 0}


## 8. Pydantic Model Support

Handle Pydantic models with `.model_dump()` method.

In [21]:
# Try importing Pydantic (skip if not available)
try:
    from pydantic import BaseModel

    class User(BaseModel):
        name: str
        age: int
        email: str | None = None

    user = User(name="Alice", age=30)

    # Default - .model_dump() is called
    result = to_dict(user)
    print(f"Pydantic model: {result}")

    # Prioritize model_dump (same result in this case)
    result_priority = to_dict(user, prioritize_model_dump=True)
    print(f"Prioritize model_dump: {result_priority}")

except ImportError:
    print("Pydantic not available - skipping Pydantic examples")

Pydantic model: {'name': 'Alice', 'age': 30, 'email': None}
Prioritize model_dump: {'name': 'Alice', 'age': 30, 'email': None}


## 9. Custom Parsers

Provide custom parsing functions for string inputs.

In [22]:
import json


# Custom parser that adds metadata
def custom_parser(s: str) -> dict:
    data = json.loads(s)
    data["_parsed"] = True
    return data


json_str = '{"name": "Test", "value": 123}'
result = to_dict(json_str, parser=custom_parser)
print(f"Custom parser result: {result}")
print(f"Has _parsed flag: {'_parsed' in result}")

Custom parser result: {'name': 'Test', 'value': 123, '_parsed': True}
Has _parsed flag: True


## 10. Error Handling

Control error handling with the `suppress` parameter.

In [23]:
# Invalid input with suppress=True → empty dict
class UnconvertibleObj:
    def __iter__(self):
        raise ValueError("Cannot iterate")


result = to_dict(UnconvertibleObj(), suppress=True)
print(f"Suppressed error: {result}")

Suppressed error: {}


In [24]:
# Empty string always returns {}
result = to_dict("", suppress=False)
print(f"Empty string: {result}")

Empty string: {}


In [25]:
# suppress=False → raises exceptions
class UnconvertibleObj:
    def __iter__(self):
        raise ValueError("Cannot iterate")


try:
    result = to_dict(UnconvertibleObj(), suppress=False)
except Exception as e:
    print(f"Exception raised: {type(e).__name__}: {e}")

## 11. Complex Real-World Example

Combine multiple features in a realistic scenario.

In [26]:
@dataclass
class Config:
    environment: str
    settings: dict


# Complex nested structure
complex_data = {
    "api_response": '{"status": "success", "data": {"count": 42}}',
    "config": Config(environment="production", settings={"debug": False, "timeout": 30}),
    "tags": {"python", "api", "production"},
    "items": [("apple", 1), ("banana", 2)],
    "nested_json": {"user": '{"name": "Alice", "role": "admin"}'},
}

result = to_dict(complex_data, recursive=True, recursive_python_only=False)

print("Complex conversion result:")
print(f"  api_response (parsed): {result['api_response']}")
print(f"  config (dataclass→dict): {result['config']}")
print(f"  tags (set→dict): {result['tags']}")
print(f"  items (list of tuples→dict): {result['items']}")
print(f"  nested_json.user (parsed): {result['nested_json']['user']}")

Complex conversion result:
  api_response (parsed): {'status': 'success', 'data': {'count': 42}}
  config (dataclass→dict): {'environment': 'production', 'settings': {'debug': False, 'timeout': 30}}
  tags (set→dict): {'production', 'api', 'python'}
  items (list of tuples→dict): [('apple', 1), ('banana', 2)]
  nested_json.user (parsed): {'name': 'Alice', 'role': 'admin'}


## 12. Type Preservation

Container types are preserved during recursive processing.

In [27]:
# Lists, tuples, sets, dicts preserved
data = {
    "list_val": [1, 2, 3],
    "tuple_val": (4, 5, 6),
    "set_val": {7, 8, 9},
    "dict_val": {"a": 1, "b": 2},
}


# Create wrapper to force conversion
@dataclass
class Container:
    data: dict


container = Container(data=data)
result = to_dict(container, recursive=True)

print(f"list type preserved: {type(result['data']['list_val']).__name__}")
print(f"tuple type preserved: {type(result['data']['tuple_val']).__name__}")
print(f"set type preserved: {type(result['data']['set_val']).__name__}")
print(f"dict type preserved: {type(result['data']['dict_val']).__name__}")

list type preserved: list
tuple type preserved: tuple
set type preserved: set
dict type preserved: dict


## Summary Checklist

**to_dict Essentials:**
- ✅ Converts dicts, lists, tuples, sets, None to dictionaries
- ✅ Auto-parses JSON strings (with fuzzy parsing option)
- ✅ Handles dataclasses via `dataclasses.asdict()`
- ✅ Converts enum classes to member dictionaries
- ✅ Supports objects with `.to_dict()`, `.dict()`, `.model_dump()`, `__dict__`
- ✅ Recursive processing with depth control (max 10 levels)
- ✅ Custom parser support for string inputs
- ✅ Error suppression (returns `{}` on failure)
- ✅ Type preservation during recursive processing
- ✅ Pydantic model support with `.model_dump()` prioritization

**Key Parameters:**
- `recursive`: Enable deep conversion (default: False)
- `max_recursive_depth`: Limit recursion depth (default: 5, max: 10)
- `recursive_python_only`: Only recurse into builtins (default: True)
- `fuzzy_parse`: Use fuzzy JSON parsing (default: False)
- `suppress`: Return `{}` on errors (default: False)
- `parser`: Custom string parser callable
- `prioritize_model_dump`: Prefer Pydantic `.model_dump()` (default: False)
- `use_enum_values`: Use enum values vs members (default: False)

**Next Steps:**
- See `to_list` for list conversions
- See `async_call` for async operations
- See `list_call` for batch operations